In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os

In [15]:
directories = ['tw/pb_ratio', "tw/price"]
parquet_files = [os.path.join(directories[1], f) for f in os.listdir(directories[1]) if f.endswith(".parquet")] 
dfs = []
parquet_files.remove("tw/price/2020.parquet")
parquet_files.remove("tw/price/2021.parquet")

In [17]:
print(parquet_files)
for file in parquet_files:
    df = pd.read_parquet(file)
    dfs.append(df)
df_concat = pd.concat(dfs)


['tw/price/2023.parquet', 'tw/price/2022.parquet', 'tw/price/2024.parquet']


In [33]:
values_dict = {
    "cl": "close",
    "op": "open",
    "vol(turnover)": "volume",
    'vol(volume)': "volume(share)",
    "yield": "yield",
    "pe_ratio": "pe_ratio",
    "pb_ratio": "pb_ratio",
}
values = 'vol(turnover)'
pivoted = df_concat.pivot(index='da', values=values, columns="ticker")
pivoted.replace("--", None, inplace=True)
pivoted = pivoted.ffill()
pivoted = pivoted.astype(float)
pivoted.astype(float).to_parquet(f"tw/pdata/{values_dict[values]}.parquet")
pivoted_pct = pivoted.pct_change().dropna(how="all")
pivoted_pct.to_parquet(f'tw/pdata/{values_dict[values]}_pct.parquet')